# Utilities

> A set of functions that I commonly use in notebooks

In [ ]:
#| default_exp utils

In [ ]:
#| export
import fastcore
import functools
from nbdev.showdoc import *
import nptyping
import numpy as np

In [ ]:
#| export
def thread_macro(current_value, *funcs, identifier="self"):
    """Pipes current_value through each function in funcs.

    Each element in funcs is either a function or a list/tuple containing
    a function followed by its other arguments.
    This function imitates the Clojure as-if threading macro.

    Notes: By default current_value is threaded as the first argument of the
    function call. Yet, one can use the syntax [func, arg1, "self", arg2] (or
    (func, arg1, "self", arg2)) so that current_value will instead be threaded
    in whatever place "self" would be. If you need to, you can set this "self"
    identifier to a different value.
    """

    for func in funcs:
        if isinstance(func, (list, tuple)):
            place = 0
            for i, el in enumerate(func[1:]):
                if el == identifier:
                    place = i
                    func = [el for el in func if el != identifier]
            func, args1, args2 = func[0], func[1:place + 1], func[place + 1:]
            current_value = func(*args1, current_value, *args2)
        else:
            current_value = func(current_value)
    return current_value

In [ ]:
#| export
def assoc(m:dict, *kargs):
    "Add every two elements in kargs as a new key-value item in dictionary `m`."
    return {**m, **dict(zip(kargs[::2],
                            kargs[1::2]))}

In [ ]:
#| export
def get(m:dict, k:str):
    "Get attribute k from dictionary m."
    return m.get(k)

In [ ]:
#| export
def broadcast_concatenate_axes(ax1, ax2):
    """Broadcast both numpy axes and concatenate along last dimension"""
    ax1new = ax1
    for _ in range(np.ndim(ax2) - 1):
        ax1new = ax1new[..., None, :]
    ax2new = ax2
    for _ in range(np.ndim(ax1) - 1):
        ax2new = ax2new[None, ..., :]
    ax1new = np.broadcast_to(ax1new,
                             (*ax1.shape[:-1], *ax2.shape[:-1], ax1.shape[-1]))
    ax2new = np.broadcast_to(ax2new,
                             (*ax1.shape[:-1], *ax2.shape[:-1], ax2.shape[-1]))
    ax = np.concatenate((ax1new, ax2new), axis=-1)
    return ax

In [ ]:
#| export
def build_grid_from_axes(axes:list, # Each axis in axes gives an array of values that should be repeated for each value in the other axes. Primitive types and lists of primitive types are first promoted to numpy arrays.
                         override:bool=False, # whether to build the grid if it is very large
                        ) -> np.ndarray: # A 2D numpy array with all combinations of elements specified in axes
    """Build a numpy array with all combinations of elements specified in axes."""

    dtypes = (float, int, bool, str)
    for i, axis in enumerate(axes):
        condition = isinstance(axis, dtypes) or all(
            isinstance(el, dtypes) for el in list(axis))
        axes[i] = np.array([axis]).T if condition else axis
    final_size = np.prod([axis.shape[0] for axis in axes])
    if (final_size > 5*10**6) & (not override):
        raise ValueError(f"""Your axes imply you want to create a grid with {final_size} > 5 million rows!
        If you're confident you can do this without crashing your computer, pass override=True to this function.""")
    tensor = functools.reduce(broadcast_concatenate_axes, axes)
    return tensor.reshape((-1, tensor.shape[-1]))

In [ ]:
show_doc(build_grid_from_axes)

---

[source](https://github.com/PaoloBova/gh-pages-example/blob/main/gh_pages_example/utils.py#L70){target="_blank" style="float:right; font-size:smaller"}

### build_grid_from_axes

>      build_grid_from_axes (axes:list, override:bool=False)

Build a numpy array with all combinations of elements specified in axes.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| axes | list |  | Each axis in axes gives an array of values that should be repeated for each value in the other axes. Primitive types and lists of primitive types are first promoted to numpy arrays. |
| override | bool | False | whether to build the grid if it is very large |
| **Returns** | **ndarray** |  | **A 2D numpy array with all combinations of elements specified in axes** |

We can build simple grids

In [ ]:
build_grid_from_axes([range(2), range(3)])

array([[0, 0],
       [0, 1],
       [0, 2],
       [1, 0],
       [1, 1],
       [1, 2]])

Or build more arbitrary grids

In [ ]:
build_grid_from_axes([[0.1, 0.7, 0.8], [5, 4]])

array([[0.1, 5. ],
       [0.1, 4. ],
       [0.7, 5. ],
       [0.7, 4. ],
       [0.8, 5. ],
       [0.8, 4. ]])

`build_grid_from_axes` throws an error if you try to build a grid which has > 5 million rows. This error should help you avoid accidentally crashing your computer with unreasonably large grids.

In [ ]:
with fastcore.test.ExceptionExpected(ex=ValueError):
    build_grid_from_axes([range(500000), range(200000)], override=False)

In [ ]:
#| export
def model_builder(saved_args:dict, # a dictionary containing parameters we want to vary or hold fixed
                  exclude_args:list[str]=[], # a list of arguments that should be returned as they are
                  override:bool=False, # whether to build the grid if it is very large
                  drop_args:list[str]=['override', 'exclude_args', 'drop_args'], # a list of arguments to drop from the final result
                 ) -> dict: # A dictionary containing items for the desired models
    """Build models for all combinations of the valules in `saved_args`."""
    axes_args1 = {k: np.array(v)
                  for k,v in saved_args.items()
                  if (isinstance(v, np.ndarray)
                      or (isinstance(v, list)
                          and all(isinstance(el, float) for el in v)))}
    axes_args2 = {k: np.arange(v["start"], v["stop"], v["step"])
                  for k,v in saved_args.items()
                  if (isinstance(v, dict)
                      and v.keys() == {"start", "stop", "step"})}
    axes_args = {**axes_args1, **axes_args2}
    grid = build_grid_from_axes(list(axes_args.values()),
                                override=override)
    models = {}
    # add grid parameters first
    for i, arg in enumerate(axes_args.keys()):
        models[arg] = grid[:, i]
    # add fixed parameters next
    for arg, v in saved_args.items():
        if arg not in (exclude_args
                       + drop_args
                       + list(axes_args.keys())):
            models[arg] = np.array([v for _ in grid])
    # add extra variables
    for arg in exclude_args:
        if (arg in saved_args.keys()
            and arg not in drop_args):
            models[arg] = saved_args[arg]
    return models

In [ ]:
show_doc(model_builder)

---

[source](https://github.com/PaoloBova/gh-pages-example/blob/main/gh_pages_example/utils.py#L88){target="_blank" style="float:right; font-size:smaller"}

### model_builder

>      model_builder (saved_args:dict, exclude_args:list[str]=[],
>                     override:bool=False, drop_args:list[str]=['override',
>                     'exclude_args', 'drop_args'])

Build models for all combinations of the valules in `saved_args`.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| saved_args | dict |  | a dictionary containing parameters we want to vary or hold fixed |
| exclude_args | list | [] | a list of arguments that should be returned as they are |
| override | bool | False | whether to build the grid if it is very large |
| drop_args | list | ['override', 'exclude_args', 'drop_args'] | a list of arguments to drop from the final result |
| **Returns** | **dict** |  | **A dictionary containing items for the desired models** |

## Clojure multimethods in python

**Disclaimer:** Unlike the code above, this code is not my invention. All credit goes to Adam Bard for coming up with this (and Guido for writing an earlier implementation). Adam Bard made this code freely available at https://adambard.com/blog/implementing-multimethods-in-python/.

Alternatively, this package exists but only works based on type hints: https://pypi.org/project/multimethod/#description.
In my opinion, the clojure dispatch function approach is far more versatile. Big thanks to Adam Bard for implementing this.

In [ ]:
#| export
def multi(dispatch_fn):
    def _inner(*args, **kwargs):
        return _inner.__multi__.get(
            dispatch_fn(*args, **kwargs),
            _inner.__multi_default__
        )(*args, **kwargs)
    
    _inner.__dispatch_fn__ = dispatch_fn
    _inner.__multi__ = {}
    _inner.__multi_default__ = lambda *args, **kwargs: None  # Default default
    return _inner

In [ ]:
#| export
def method(dispatch_fn, dispatch_key=None):
    def apply_decorator(fn):
        if dispatch_key is None:
            # Default case
            dispatch_fn.__multi_default__ = fn
        else:
            dispatch_fn.__multi__[dispatch_key] = fn
        return dispatch_fn
    return apply_decorator

### Examples and tests

In [ ]:
#| export
@multi
def area(shape):
    return shape.get('type')

@method(area, 'square')
def area(square):
    return square['width'] * square['height']

@method(area, 'circle')
def area(circle):
    return circle['radius'] ** 2 * 3.14159

@method(area)
def area(unknown_shape):
    raise Exception("Can't calculate the area of this shape")

In [ ]:
#| export
fastcore.test.test_eq(area({'type': 'square', 'width': 1, 'height': 1}), 1)
fastcore.test.test_close(area({'type': 'circle', 'radius': 0.5}), 0.7853975)
with fastcore.test.ExceptionExpected():
    area({'type': 'rhombus'})

A method for creating all strategy profile codes for a set of models.

In [ ]:
#| export
def create_all_profiles(models):
    """Create all strategy profiles for the set of models."""
    sector_strategies = models.get('sector_strategies', {})
    allowed_sectors = models.get('allowed_sectors', {})
    n_players = models.get('n_players', len(allowed_sectors))
    n_strategies = models.get('n_strategies',
                              [len(v) for v in sector_strategies.values()])
    n_strategies_total = np.sum(n_strategies)
    n_strategies_total += 1  # Add null sector.
    n_profiles = n_strategies_total ** n_players
    strategy_axis = np.arange(n_strategies_total)[:, None]
    grid = build_grid_from_axes([strategy_axis for _ in range(n_players)])
    profiles = []
    for row in grid:
        profile = "-".join(map(str, row))
        profiles.append(profile)
    fastcore.test.test_eq(len(profiles), n_profiles)
    return {**models, "profiles": profiles}

##### Tests for `create_all_profiles`

In [ ]:
# | export
result = create_all_profiles({"n_players": 2, "n_strategies": 2})
fastcore.test.test_eq(result['profiles'],
                      ['0-0', '0-1', '0-2',
                       '1-0', '1-1', '1-2',
                       '2-0', '2-1', '2-2'])

result = create_all_profiles({"n_players": 2, "n_strategies": [2, 1]})
fastcore.test.test_eq(result['profiles'],
                      ["0-0", "0-1", "0-2", "0-3",
                       "1-0", "1-1", "1-2", "1-3",
                       "2-0", "2-1", "2-2", "2-3",
                       "3-0", "3-1", "3-2", "3-3", ])

fastcore.test.test_eq(create_all_profiles({"n_players": 2,
                                           "n_strategies": [2, 2]})['profiles'],
                      create_all_profiles({"n_players": 2,
                                          "n_strategies": 4})['profiles'])

models = {"allowed_sectors": {"P1": ["S1"],
                              "P2": ["S2"]},
          "sector_strategies": {"S1": [1, 2],
                                "S2": [3, 4]}}
fastcore.test.test_eq(create_all_profiles(models)['profiles'],
                      create_all_profiles({"n_players": 2,
                                          "n_strategies": 4})['profiles'])


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()